# How to read data files?
This notebook describes or points to modules for reading data in different file formats and from different sources.

| Format        | Tool           | 
| ------------- |:-------------:| 
| NetCDF        | [NCDatasets.jl](https://github.com/Alexander-Barth/NCDatasets.jl) | 
| ODV           | [ODVspreadsheet.jl](https://github.com/gher-ulg/divand.jl/blob/master/src/ODVspreadsheet.jl) |
| GEBCO bathymetry | [load_mask.jl](https://github.com/gher-ulg/divand.jl/blob/master/src/load_mask.jl)|
| Big files     | [load_obs.jl](https://github.com/gher-ulg/divand.jl/blob/master/src/load_obs.jl) |
| Mat files     | [MAT.jl](https://github.com/JuliaIO/MAT.jl)

In [3]:
using divand

## NetCDF

The 2 main modules to read netCDF are:
1. [NetCDF.jl](https://github.com/JuliaGeo/NetCDF.jl)
2. [NCDatasets.jl](https://github.com/Alexander-Barth/NCDatasets.jl)

An example is provided in this [notebook](./03-netCDF.ipynb).

## ODV spreadsheet
ODV spreadsheets constitute one of the standard formats in [SeaDataCloud](https://www.seadatanet.org/). In `DIVAnd`, we provide a tool called [ODVspreadsheet.jl](https://github.com/gher-ulg/divand.jl/blob/master/src/ODVspreadsheet.jl) designed to read such format.

An example is provided in this [notebook](./09-ODV-data-import.ipynb).

## Big files
The so-called big files are intermediate files using by DIVA and DIVAnd. The format is rather simple: a tab-separated file containing the following variables:
1. longitude,
2. latitutde,
3. field value (e.g., temperature, salinity, chlorophyll concentration, ...), 
4. depth,
5. time,
6. measurement identifier.

In the module [load_obs.jl](https://github.com/gher-ulg/divand.jl/blob/master/src/load_obs.jl), the function `loadbigfile` allows the reading of such file format.    
In the next cell we download a *big file* containing salinity measurements (also used in other examples) and read it using `loadbigfile`.

In [6]:
fname = "Salinity.bigfile"
if !isfile(fname)
    download("https://b2drop.eudat.eu/s/Bv9Fj0YGC0zp2vn/download",fname)
else
    info("Data file already downloaded")
end
value,lon,lat,depth,time,ids = loadbigfile(fname);

## Bathymetry
The General Bathymetric Chart of the Oceans [GEBCO](https://www.gebco.net/) (in netCDF) is directly read with `DIVAnd` using the function `load_bath`.  

First make sure we have a bathymetry file.

In [4]:
bathname = "gebco_30sec_16.nc"
if !isfile(bathname)
    download("https://b2drop.eudat.eu/s/ACcxUEZZi6a4ziR/download",bathname)
else
    info("Bathymetry file already downloaded")
end

Then we have to define the grid on which we need the bathymetry and apply the function.

In [5]:
lonr = -10:0.5:36.
latr = 37:0.5:48
bx,by,b = load_bath(bathname,true,lonr,latr);

`bx` and `by` are the same as lonr and latr.    
`b` contains the bathymetry values.

A complete example is provided in this [notebook](./06-topography.ipynb). 

## Mat files
The [MAT](https://github.com/JuliaIO/MAT.jl) module was designed to read the files created by MATLAB (.mat extension).      We use the same .mat file as in [04-OI-variational-analysis-introduction](./04-OI-variational-analysis-introduction.ipynb).

In [1]:
using MAT

INFO: Recompiling stale cache file /home/ctroupin/.julia/v0.6/lib/v0.6/Blosc.ji for module Blosc.
INFO: Recompiling stale cache file /home/ctroupin/.julia/v0.6/lib/v0.6/MAT.ji for module MAT.


In [3]:
matfile = "dan_field.mat"
mf = matopen(matfile)

MAT.MAT_v5.Matlabv5File(IOStream(<file dan_field.mat>), false, #undef)

We can get a list of the variables stored in the file:

In [4]:
varnames = names(mf)

Base.KeyIterator for a Dict{String,Int64} with 3 entries. Keys:
  "f"
  "Fe"
  "F"

and to load one of them, use

In [6]:
var1 = read(mf, "f");
@show sizeof(var1);

sizeof(var1) = 20000


When we're done, don't forget to close the file.

In [7]:
close(mf)